In [1]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import AutoModel, AutoProcessor, BeitForMaskedImageModeling
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:
file_path = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/AllCarabids.csv"
insect_df = pd.read_csv(file_path, sep="\t")
insect_df.rename(columns={"filepath": "ImageFilePath", "genus" : "Genus", "class": "ScientificName"}, inplace=True)
insect_df['ImageFilePath'] = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/" + insect_df['ImageFilePath']
df = insect_df[['ImageFilePath', 'ScientificName']]
df = df.fillna("Unknown")
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "microsoft/beit-base-patch16-224"

model = AutoModel.from_pretrained(model_name).eval().to("cuda" if torch.cuda.is_available() else "cpu")

processor = AutoProcessor.from_pretrained(model_name)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
def extract_features(image_path):

    image = Image.open(image_path).convert("RGB")
    
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)

    features = outputs.last_hidden_state.mean(dim=1)

    return features.cpu().numpy().flatten()
    

In [5]:
df_known = df[df["ScientificName"] != "Unknown"]

df_unknown = df[df["ScientificName"] == "Unknown"]


X_known = np.vstack([extract_features(img) for img in tqdm(df_known["ImageFilePath"], desc="Processing ...")])


le = LabelEncoder()

y_known = le.fit_transform(df_known["ScientificName"])

df_indices_known = df_known.index


X_train, X_test_known, y_train, y_test_known, train_idx, test_idx_known = train_test_split(
    X_known, y_known, df_indices_known, test_size=0.2, random_state=42
)


X_test_unknown = np.vstack([extract_features(img) for img in tqdm(df_unknown["ImageFilePath"], desc="Processing Unknown ...")])

test_idx_unknown = df_unknown.index

y_test_unknown = np.full(len(X_test_unknown), -1)


X_test = np.vstack([X_test_known, X_test_unknown])

y_test = np.concatenate([y_test_known, y_test_unknown])

test_idx = np.concatenate([test_idx_known, test_idx_unknown])

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")



test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)



Processing Unknown ...: 100%|██████████| 4355/4355 [01:33<00:00, 46.49it/s]


Train set: (77224, 768), Test set: (23661, 768)


In [6]:
models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),   
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=42)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    # print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precisio

In [7]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
19596,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela repanda,Cicindela willistoni,Cicindelidia rufiventris,Cicindela maritima,Cicindelidia rufiventris
18754,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Agonum aeruginosum,Calosoma calidum,Agonum aeruginosum,Amara anthobia,Agonum aeruginosum


In [8]:
test_df.to_csv("/users/PAS2136/rayees/CV4A - Benchmarking/AllCarabids/8.BeIT-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/CV4A - Benchmarking/AllCarabids/8.BeIT-linear-probing-species-metrics.csv", index=False)